# Linear Regression

This section builds a linear regression model which will predict crop yeilds in hte region by looking at average temp, rainfall and humidity.

In [2]:
import torch
import numpy as np

In [10]:
# Input values of temp, rainfall, humidity in a numpy array
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [11]:
# Target  value of crop yield, apples,oranges.
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [12]:
#Converting inputs and targets to tensors
tensor_inputs = torch.from_numpy(inputs)
tensor_targets = torch.from_numpy(targets)
print(tensor_inputs,tensor_targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


### Linear regression model from scratch

Defining weights and biases

In [13]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)

The model simply performs matrix multiplication. 

Note `@ operator` represents matrix multiplication in pytorch. and a `.t` represents transpose of a matrix.

In [14]:
def model(x):
    return x @ w.t() + b

### Perdications Using the model

In [16]:
predictions = model(tensor_inputs)
print(predictions)

tensor([[ 50.7197, -11.8145],
        [ 57.0893, -25.6647],
        [ 78.5908,  36.1860],
        [ 67.1980, -37.7570],
        [ 39.0483, -19.6343]], grad_fn=<AddBackward0>)


Comparing the predictions with actual target.

In [17]:
print(tensor_targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Difference is calculated between `predictions` and `targets`. The `.numel()` method gives number of elements in a tensor, and `.sum()` method gives the sum of elements in a tensor. Here the mean squared error (MSE) is being calculated.

In [20]:
diff = predictions - tensor_targets
torch.sum(diff * diff)/ diff.numel()

tensor(6528.3765, grad_fn=<DivBackward0>)

# Loss Function

Loss function is used to evaluate how well the model is perofrming. In this case we will use MSE loss.

In [22]:
def mse(t1,t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [23]:
loss = mse(predictions, tensor_targets)
print(loss)

tensor(6528.3765, grad_fn=<DivBackward0>)


## Computing Gradients

In [24]:
loss.backward()

Gradient for weights

In [25]:
print(w)
print(w.grad)

tensor([[ 0.7635,  0.4317, -0.7828],
        [-0.2855,  1.1196, -1.5345]], requires_grad=True)
tensor([[-1175.8833, -2413.7222, -1381.0737],
        [-8622.5488, -9224.9170, -5942.0371]])


The loss is a quadratic function and the object is to find the set of weights and biases where the loss is lowest. Now the weight and baises gradients are reset to 0. This is done because the `.backward()` method accumulates the loss.

In [30]:
w.grad.zero_()
b.grad.zero_()
print(w.grad, b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])tensor([0., 0.])


### Adjusting weights and biases using gradient descent

In [31]:
predictions = model(tensor_inputs)
print(predictions)
loss = mse(predictions, tensor_targets)
print(loss)

tensor([[ 50.7197, -11.8145],
        [ 57.0893, -25.6647],
        [ 78.5908,  36.1860],
        [ 67.1980, -37.7570],
        [ 39.0483, -19.6343]], grad_fn=<AddBackward0>)
tensor(6528.3765, grad_fn=<DivBackward0>)


Calculating the gradients for the loss.

In [32]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-1175.8833, -2413.7222, -1381.0737],
        [-8622.5488, -9224.9170, -5942.0371]])
tensor([ -17.6708, -103.7369])


Adjust the weights and biases using the gradients. 
Here `torch.no_grad()` is beign used so that the gradients don't change during the computations of new weights and baises.

In [33]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [34]:
print(w,b)

tensor([[ 0.7753,  0.4558, -0.7690],
        [-0.1992,  1.2119, -1.4750]], requires_grad=True)tensor([-0.2795, -0.0063], requires_grad=True)


In [35]:
predictions = model(tensor_inputs)
loss = mse(predictions, tensor_targets)
print(loss)

tensor(4671.5146, grad_fn=<DivBackward0>)


The loss has thus been minimized.

Now, we train for multiple epochs, thats is multiple for loops and check the result.

In [38]:
for i in range(10000):
    predictions = model(tensor_inputs)
    loss = mse(predictions, tensor_targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

print(predictions)
print(tensor_targets)
print(loss)

tensor([[ 57.1181,  70.3101],
        [ 82.2364, 100.6692],
        [118.7008, 132.9589],
        [ 21.0875,  37.0153],
        [101.9128, 119.1375]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])
tensor(0.5109, grad_fn=<DivBackward0>)
